In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from matplotlib.ticker import EngFormatter
from matplotlib.colors import ListedColormap
import matplotlib as mpl
import re
import numpy as np
from jinja2 import Template
from subprocess import check_call
import concurrent.futures
import multiprocessing as mp

In [ ]:
# Reading Pad Documentation csv file to extract parameters
doc_df = pd.read_csv("pad_documentation/Pad_documentation _for _SKY130_ MPW_Manufacturing.csv")
nfet_01v8_df = doc_df.loc[doc_df["New Style Name"] == "nfet_01v8"]

In [ ]:
# curve_type must be one of ['ID_VDS', 'ID_VGS']
curve_type = "ID_VDS"

In [ ]:
# Generating sim data
def simulate_device(netlist_path):
    check_call(f"ngspice -b {netlist_path} -o output.log  ", shell= True)

workers_count = 2 * mp.cpu_count()
nmos_netlist = "templates/nfet_01v8_template.cir"
combs = list()

if curve_type == "ID_VDS":
    sim_cmd = "DC vds 0 1.8 0.05"

elif curve_type == "ID_VGS":
    sim_cmd = "DC vgs 0 1.8 0.05"

with concurrent.futures.ProcessPoolExecutor(max_workers=workers_count) as executor:
    for i, row in nfet_01v8_df.iterrows():
        sim_data = row["Description"]
        subs = [';', 'nshort', 'in DNW', 'contact-gate=\d+.\d+um']
        for element in subs:
            sim_data = re.sub(element, ' ' ,sim_data)

        # get pins loc for each file
        pin1 = int()
        pin2 = int()
        for c in row.index:
            if row[c] == "s":
                pin1 = int(re.sub("Pin ", '', c))
            elif row[c] == "d":
                pin2 = int(re.sub("Pin ", '', c))
        
        # get width, length, multiplier
        width = re.findall("w=\d*.\d*", sim_data)[0]
        width = re.sub("=", "", width)
        length = re.findall("l=\d*.\d*", sim_data)[0]
        length = re.sub("=", "", length)
        multiplier = re.findall("m=\d*", sim_data)[0]
        multiplier = re.sub("=", "", multiplier)

        combs.append((width, length, multiplier, pin1, pin2))
        # naming circuit, csv files
        file_name = f"nfet_01v8_{width}_{length}_{multiplier}_{pin1}_{pin2}"
        netlist_path = f"{file_name}.cir"
        csv_path = f"{file_name}.csv"

        with open(nmos_netlist) as f:
                    tmpl = Template(f.read())
                    with open(netlist_path, "w") as netlist:
                        netlist.write(
                            tmpl.render(
                                csv_path=csv_path,
                                dimensions=sim_data,
                                sim_cmd=sim_cmd,
                            )
                        )
        
        executor.submit(simulate_device, netlist_path)

In [ ]:
# clean unused files
check_call("rm -f nfet_*.cir", shell=True)
check_call("rm -f output.log", shell=True)

In [18]:
# widgets settings 
w_sweep = list()
l_sweep = list()
m_sweep = list()
s_pin = list()
d_pin = list()
vsb = [0, 0.9]
for comb in combs:
    w_sweep.append(float(comb[0][1: ]))
    l_sweep.append(float(comb[1][1: ]))
    m_sweep.append(float(comb[2][1: ]))
    s_pin.append(float(comb[3]))
    d_pin.append(float(comb[4]))

w = widgets.Dropdown(
    options=w_sweep,
    value=w_sweep[0],
    description='Width:',
    disabled=False,
)
l = widgets.Dropdown(
    options=l_sweep,
    value=l_sweep[0],
    description='Length:',
    disabled=False,
)
m = widgets.Dropdown(
    options=m_sweep,
    value=m_sweep[0],
    description='Multiplier:',
    disabled=False,
)
source = widgets.Dropdown(
    options=s_pin,
    value=s_pin[0],
    description='Source pin:',
    disabled=False,
)
drain = widgets.Dropdown(
    options=d_pin,
    value=d_pin[0],
    description='Drain pin:',
    disabled=False,
)
vsb = widgets.Dropdown(
    options=vsb,
    value=vsb[0],
    description='Drain pin:',
    disabled=False,
)

In [ ]:
blue_range =  ListedColormap(mpl.cm.Blues(np.linspace(0.3, 0.8, 256)))
red_range = ListedColormap(mpl.cm.Reds(np.linspace(0.3, 0.8, 256)))

# plotting data:
def plot(W, L, VSB):
    
    
# interactive mode
widgets.interact(plot, w=w, l=l, vsb=vsb, m=m, source=source, drain=drain)